---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [1]:
import pandas as pd
import re
#Keep in mind that this is an Excel file, and not a comma separated values file. 
#Also, make sure to exclude the footer and header information from the datafile. 
#The first two columns are unneccessary, so you should get rid of them, and you should 
#change the column labels so that the columns are:
#['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
energy = pd.read_excel('Energy Indicators.xls', skiprows=17, skip_footer= 38, na_values=["..."])
energy = energy.drop(['Unnamed: 0', 'Unnamed: 1'], axis = 1)\
.rename(columns = {'Unnamed: 2' : 'Country', 'Petajoules': 'Energy Supply', 
                   'Gigajoules' : 'Energy Supply per Capita', '%' : '% Renewable'})

#Convert Energy Supply to gigajoules (there are 1,000,000 gigajoules in a petajoule). 
#For all countries which have missing data (e.g. data with "...") make sure this is reflected as np.NaN values.
energy['Energy Supply'] = energy['Energy Supply'].apply(lambda x: x*1e6)

#There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these,
#e.g.
#'Bolivia (Plurinational State of)' should be 'Bolivia',
#'Switzerland17' should be 'Switzerland'.
energy.Country = energy.Country.apply(lambda x: re.split("(\s\()|[0-9]", x)[0])

#Rename the following list of countries (for use in later questions):
fixed_names = {'Republic of Korea': 'South Korea', 'United States of America' : 'United States', 
               "United Kingdom of Great Britain and Northern Ireland" : "United Kingdom",
               "China, Hong Kong Special Administrative Region" : "Hong Kong"} 
energy.Country = energy.Country.apply(lambda x: fixed_names[x] if x in fixed_names.keys() else x)

In [2]:
#Next, load the GDP data from the file world_bank.csv, which is a csv containing countries' 
#GDP from 1960 to 2015 from World Bank. Call this DataFrame GDP.
#Make sure to skip the header, and rename the following list of countries:
#"Korea, Rep.": "South Korea", 
#"Iran, Islamic Rep.": "Iran",
#"Hong Kong SAR, China": "Hong Kong"

GDP = pd.read_csv('world_bank.csv', skiprows=4)
fixed_names = {'Korea, Rep.' : 'South Korea', 'Iran, Islamic Rep.' : 'Iran', 'Hong Kong SAR, China' : 'Hong Kong'}
GDP['Country Name'] = GDP['Country Name'].apply(lambda x: fixed_names[x] if x in fixed_names.keys() else x)

In [3]:
#Finally, load the Sciamgo Journal and Country Rank data for Energy Engineering and 
#Power Technology from the file scimagojr-3.xlsx, which ranks countries based on their 
#journal contributions in the aforementioned area. Call this DataFrame ScimEn.
ScimEn = pd.read_excel('scimagojr-3.xlsx')
ScimEn.head()

,Rank,Country,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
0,1,China,127050,126767,597237,411683,4.70,138
1,2,United States,96661,94747,792274,265436,8.20,230
2,3,Japan,30504,30287,223024,61554,7.31,134
3,4,United Kingdom,20944,20357,206091,37874,9.84,139
4,5,Russian Federation,18534,18301,34266,12422,1.85,57


In [4]:
#Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). 
#Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15).
#The index of this DataFrame should be the name of the country, and the columns should be 
#['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 
#'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', 
#'2010', '2011', '2012', '2013', '2014', '2015'].
#This function should return a DataFrame with 20 columns and 15 entries.
def answer_one():
    answer = energy.merge(GDP[['Country Name'] + list(GDP.columns[-10:].values)] \
                      ,how = 'inner',left_on= 'Country', right_on = 'Country Name')
    answer.drop('Country Name', axis =1, inplace= True)
    answer = pd.merge(ScimEn[ScimEn.Rank < 16], answer, how = 'inner', on = 'Country')
    answer.set_index('Country', inplace=True)
    return answer

### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [5]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [6]:
def answer_two():
    outer = energy.merge(GDP[['Country Name'] + list(GDP.columns[-10:].values)] \
                      ,how = 'outer',left_on= 'Country', right_on = 'Country Name')
    outer.drop('Country Name', axis =1, inplace= True)
    outer_2 = pd.merge(ScimEn, outer, how = 'outer', on = 'Country')
    inner = ScimEn.merge(energy.merge(GDP[['Country Name'] + list(GDP.columns[-10:].values)] , 
                                      left_on= 'Country', right_on = 'Country Name').drop('Country Name', axis =1),
                        on = 'Country')
    return len(outer_2)-len(inner)
answer_two()

160

<br>

Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [7]:
import numpy as np
def answer_three():
    Top15 = answer_one()
    return Top15[Top15.columns[-10:]].apply(np.mean,axis =1).sort_values(ascending=False)
answer_three()

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
dtype: float64

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [8]:
def answer_four():
    Top15 = answer_one()
    country_6 = answer_three().index[5]
    return (Top15.loc[country_6,'2015']-Top15.loc[country_6,'2006'])
answer_four()

246702696075.3999

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [9]:
def answer_five():
    Top15 = answer_one()
    return Top15['Energy Supply per Capita'].mean()
answer_five()

157.59999999999999

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [10]:
def answer_six():
    Top15 = answer_one()
    return (Top15['% Renewable'].sort_values().index[-1], Top15['% Renewable'].sort_values()[-1])
answer_six()

('Brazil', 69.648030000000006)

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [11]:
def answer_seven():
    Top15 = answer_one()
    Top15['ratio'] = Top15.apply(lambda x: float(x['Self-citations'])/x['Citations'], axis=1) 
    Top15 = Top15.sort_values('ratio')
    return (Top15.index[-1], Top15.ratio.values[-1])
answer_seven()

('China', 0.68931261793894216)

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [12]:
def answer_eight():
    Top15 = answer_one()
    return(Top15.assign(pop_estimate = lambda x: x['Energy Supply']/x['Energy Supply per Capita'], axis=1).
             sort_values('pop_estimate').index[-3])
answer_eight()

'United States'

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [13]:
def answer_nine():
    Top15 = answer_one()
    Top15 = Top15.assign(citable_documents_per_person = lambda x: x['Citable documents']/(x['Energy Supply']/x['Energy Supply per Capita']))
    return Top15['citable_documents_per_person'].corr(Top15['Energy Supply per Capita'])
answer_nine()

0.79400104354429457

In [14]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [15]:
# plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [16]:
def answer_ten():
    Top15 = answer_one()
    Top15['HighRenew'] = Top15.apply(lambda x: 0 if x['% Renewable'] < Top15['% Renewable'].median() else 1, axis=1)
    return Top15.sort_values('Rank')['HighRenew']
answer_ten()

Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: HighRenew, dtype: int64

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [17]:
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}

def answer_eleven():
    Top15 = answer_one()
    Top15 = Top15.assign(pop_estimate = lambda x: x['Energy Supply']/(x['Energy Supply per Capita'])).reset_index()
    Top15['Continent'] = Top15.apply(lambda x: ContinentDict[x['Country']], axis=1)  
    return Top15.groupby('Continent').pop_estimate.agg(['size', 'sum', 'mean', 'std'])
answer_eleven()

,size,sum,mean,std
Continent,,,,
Asia,5,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1,2.331602e+07,2.331602e+07,NaN
Europe,6,4.579297e+08,7.632161e+07,3.464767e+07
North America,2,3.528552e+08,1.764276e+08,1.996696e+08
South America,1,2.059153e+08,2.059153e+08,NaN


### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [18]:
def answer_twelve():
    Top15 = answer_one()
    Top15 = Top15.reset_index()
    Top15['Continent'] = Top15.apply(lambda x: ContinentDict[x['Country']], axis=1)
    Top15['Renew_bin'] = pd.cut(Top15['% Renewable'],5, labels=[1,2,3,4,5])
    return Top15.groupby(['Continent','Renew_bin']).agg('size')
answer_twelve()

Continent      Renew_bin
Asia           1            4
               2            1
Australia      1            1
Europe         1            1
               2            3
               3            2
North America  1            1
               5            1
South America  5            1
dtype: int64

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [19]:
def answer_thirteen():
    Top15 = answer_one()
    def pop_estimate(row):
        row['pop_estimate'] = float(row['Energy Supply'])/(row['Energy Supply per Capita'])
        return row
    Top15 = Top15.apply(pop_estimate,axis=1)
    return Top15['pop_estimate'].apply(lambda x: '{:,}'.format(x))
answer_thirteen()

Country
China                 1,367,645,161.2903225
United States          317,615,384.61538464
Japan                  127,409,395.97315437
United Kingdom         63,870,967.741935484
Russian Federation            143,500,000.0
Canada                  35,239,864.86486486
Germany                 80,369,696.96969697
India                 1,276,730,769.2307692
France                  63,837,349.39759036
South Korea            49,805,429.864253394
Italy                  59,908,256.880733944
Spain                    46,443,396.2264151
Iran                    77,075,630.25210084
Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
Name: pop_estimate, dtype: object

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [20]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [21]:
# plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!